In [ ]:
# Cisco Device Security Configuration Script
# This script configures basic password security on Cisco devices

# First, install netmiko if you haven't already:
# pip install netmiko

from netmiko import ConnectHandler
from getpass import getpass

def configure_cisco_security(device_info, passwords):
    """
    Configure basic security settings on a Cisco device:
    - Console password
    - VTY line passwords
    - Enable password
    - Enable secret
    - Service password-encryption
    """
    
    # Security configuration commands
    config_commands = [
        # Enable service password-encryption (encrypts ALL passwords)
        'service password-encryption',
        
        # Configure console line password
        'line console 0',
        f'password {passwords["console"]}',
        'login',
        'exit',
        
        # Configure VTY lines (remote access) password
        'line vty 0 4',
        f'password {passwords["vty"]}',
        'login',
        'exit',
        
        # Additional VTY lines if available (5-15)
        'line vty 5 15',
        f'password {passwords["vty"]}',
        'login',
        'exit',
        
        # Configure enable password (will be encrypted by service password-encryption)
        f'enable password {passwords["enable"]}',
        
        # Configure enable secret (uses stronger MD5 encryption)
        f'enable secret {passwords["secret"]}',
    ]
    
    try:
        print(f"Connecting to {device_info['host']}...")
        connection = ConnectHandler(**device_info)
        
        print("Sending security configuration...")
        output = connection.send_config_set(config_commands)
        print(output)
        
        # Save the configuration
        print("Saving configuration...")
        save_output = connection.save_config()
        print(save_output)
        
        # Verify the configuration
        print("\n--- Verifying Configuration ---")
        verify_output = connection.send_command('show running-config | include password|secret|service')
        print(verify_output)
        
        connection.disconnect()
        print("\nConfiguration complete! Device secured.")
        return True
        
    except Exception as e:
        print(f"Error: {e}")
        return False


# ============ MAIN PROGRAM ============
if __name__ == "__main__":
    
    # Device connection information
    device = {
        'device_type': 'cisco_ios',  # Change to 'cisco_nxos', 'cisco_asa', etc. if needed
        'host': '192.168.1.1',       # <-- Change to your device IP
        'username': 'admin',          # <-- Change to your username
        'password': 'current_password',  # <-- Current password to connect
        'secret': '',                 # Current enable secret (if any)
    }
    
    # New passwords to configure
    new_passwords = {
        'console': 'ConsolePass123',  # Console line password
        'vty': 'VtyPass123',          # VTY lines password  
        'enable': 'EnablePass123',    # Enable password
        'secret': 'SecretPass123',    # Enable secret (strongest)
    }
    
    # Run the configuration
    configure_cisco_security(device, new_passwords)

## Interactive Version (More Secure)
Use this version to enter passwords without hardcoding them:

In [ ]:
# Interactive version - prompts for passwords securely
from netmiko import ConnectHandler
from getpass import getpass

def interactive_setup():
    """Interactive version that prompts for all credentials"""
    
    print("=== Cisco Security Configuration Tool ===\n")
    
    # Get device connection info
    host = input("Enter device IP address: ")
    username = input("Enter username: ")
    current_password = getpass("Enter current password: ")
    
    # Get new passwords to set
    print("\n--- New Security Passwords ---")
    console_pass = getpass("Enter NEW console password: ")
    vty_pass = getpass("Enter NEW VTY password: ")
    enable_pass = getpass("Enter NEW enable password: ")
    enable_secret = getpass("Enter NEW enable secret: ")
    
    device = {
        'device_type': 'cisco_ios',
        'host': host,
        'username': username,
        'password': current_password,
    }
    
    passwords = {
        'console': console_pass,
        'vty': vty_pass,
        'enable': enable_pass,
        'secret': enable_secret,
    }
    
    # Confirm before applying
    print(f"\nReady to configure security on {host}")
    confirm = input("Proceed? (yes/no): ")
    
    if confirm.lower() == 'yes':
        configure_cisco_security(device, passwords)
    else:
        print("Cancelled.")

# Uncomment to run interactively:
# interactive_setup()

## What This Configures

| Setting | Command | Purpose |
|---------|---------|---------|
| **Service Password-Encryption** | `service password-encryption` | Encrypts ALL passwords in config (Type 7) |
| **Console Password** | `line console 0` → `password X` → `login` | Protects physical console access |
| **VTY Password** | `line vty 0 15` → `password X` → `login` | Protects Telnet/SSH remote access |
| **Enable Password** | `enable password X` | Protects privilege mode (weaker) |
| **Enable Secret** | `enable secret X` | Protects privilege mode (MD5 hash - stronger) |

⚠️ **Note:** Enable secret takes precedence over enable password. Both are included for completeness, but enable secret is the one that will be used.